In [ ]:
!pip install langchain langchain-community langchain_groq chromadb gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 110.2 MB/s et

In [ ]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [ ]:
anime  = pd.read_csv("/content/anime-filtered.csv")
anime.head()

,anime_id,Name,Score,Genres,English name,Japanese name,sypnopsis,Type,Episodes,Aired,...,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,1,"Sep 1, 2001",...,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,26,"Jul 2, 2002 to Dec 24, 2002",...,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108


In [ ]:
anime.dropna()

,anime_id,Name,Score,Genres,English name,Japanese name,sypnopsis,Type,Episodes,Aired,...,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,1,"Sep 1, 2001",...,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,26,"Jul 2, 2002 to Dec 24, 2002",...,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14916,47591,Kuro Gal ni Natta kara Shinyuu to shitemita.,6.51,"Romance, Shounen Ai",Unknown,黒ギャルになったから親友としてみた。,Shion and Rui are the dream team when it comes...,TV,Unknown,"Feb 11, 2021 to ?",...,4 min.,R+ - Mild Nudity,15231.0,7387,3636,90,768,3,79,35
14917,47614,Nu Wushen de Canzhuo Spring Festival Special,6.83,"Slice of Life, Comedy",Cooking with Valkyries Spring Festival Special,Unknown,No synopsis information has been added to this...,Special,1,"Feb 11, 2021",...,10 min.,PG - Children,4382.0,11973,540,8,51,168,18,2
14918,47616,Yakusoku no Neverland 2nd Season: Michishirube,4.81,"Mystery, Psychological, Supernatural, Thriller...",The Promised Neverland Season 2 Episode 5.5,約束のネバーランド 特別編「道標」,cap of the first season of Yakusoku no Neverla...,Special,1,"Feb 12, 2021",...,23 min.,R - 17+ (violence & profanity),10760.0,4398,13070,90,1183,8196,119,202
14924,47795,Ai Zai Xi Yuan Qian 2nd Season,6.51,"Fantasy, Romance",Immemorial Love For You 2nd Season,爱在西元前 第二季,No synopsis information has been added to this...,ONA,16,"Feb 14, 2021 to ?",...,Unknown,Unknown,13254.0,12411,463,35,40,0,7,7


In [ ]:
anime['combined_df'] =anime.apply(lambda row: f"Title: {row['Name']}. Overview : {row['sypnopsis']} Genres : {row['Genres']}", axis = 1)

In [ ]:
anime['combined_df'][0]

'Title: Cowboy Bebop. Overview : In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Beb

In [ ]:
anime[['combined_df']].to_csv('anime_updated.csv', index = False)

In [ ]:
pd.read_csv("/content/anime_updated.csv")

,combined_df
0,Title: Cowboy Bebop. Overview : In the year 20...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview : Vash the Stampede is...
3,Title: Witch Hunter Robin. Overview : ches are...
4,Title: Bouken Ou Beet. Overview : It is the da...
...,...
14947,Title: Daomu Biji Zhi Qinling Shen Shu. Overvi...
14948,Title: Mieruko-chan. Overview : ko is a typica...
14949,Title: Higurashi no Naku Koro ni Sotsu. Overvi...
14950,Title: Yama no Susume: Next Summit. Overview :...


**Data** Loader and VectorStore using **Langchain**

In [ ]:
loader = CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)
embeddings = HuggingFaceEmbeddings()
llm = ChatGroq(api_key = "gsk_NHgqXXXx3sxGksbu5u3aWGdyb3FYXj8BKec5Sk29Jn2pkD83RAtb", model = "llama-3.3-70b-versatile", temperature= 0 )

docssearch = Chroma.from_documents(texts, embeddings)

<ipython-input-8-3738cf599ba3>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-8-3738cf599ba3>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Querying Vector DB Store for movie recommendation**

In [ ]:
query = "I am looking for an animated sci-fi movie. What could you suggest to me"
docs = docssearch.similarity_search(query, k =1)

In [ ]:
docs

[Document(metadata={'row': 7557, 'source': '/content/anime_updated.csv'}, page_content='combined_df: Title: SF Saiyuuki Starzinger (Movie). Overview : Princess Aurora, the only survivor of her planet destroyed by evil mutants, is resolved to put an end to the mutant menace in the galaxy along with three enhanced human cyborgs Jesse Dart, Porkos and Aramus, she travels space in the Cosmos Queen to the Dekkos System to fight the final battle. (Source: ANN) Genres : Adventure, Mecha, Sci-Fi, Shounen')]

**Using QA Retrieval for movie recommendation**

In [ ]:
qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True)

In [ ]:
query = "Can you recommend a romance anime with a unique storyline?"
result = qa({"query": query})

<ipython-input-12-dbd1f9893e3e>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


In [ ]:
result['result']

'Based on the provided information, I would recommend "Kashimashi: Girl Meets Girl" for its unique storyline. The anime features a main character who undergoes a transformation from a boy to a girl after a bizarre twist of fate, and then navigates social life and relationships in their new form. This premise offers a fresh and interesting take on the traditional romance genre.'

In [ ]:
result['source_documents'][0]

Document(metadata={'row': 2407, 'source': '/content/anime_updated.csv'}, page_content='combined_df: Title: Natsu e no Tobira. Overview : arion is a young schoolboy who prides himself on his adherence to a philosophy he calls "rationalism". Because of his disdain for emotional display, he ignores anything remotely akin to affection. But when he\'s entangled in a romantic affair with an older courtesan, his rationalism is revealed to be little more than a cover for his own emotional immaturity. Learning to love, Marion blossoms under his older lover\'s care but unfortunately, Marion has yet to learn the true price of the affair. (Source: ANN) Genres : Drama, Romance, Shoujo, Shounen Ai')

**Prompt Enginnering**

In [ ]:
from langchain.prompts import PromptTemplate

template = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka. 
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime, 
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya. 
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}

Question: {question}
Your response:"""

PROMPT  =  PromptTemplate(
    template=template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Give me a list of comedy animes that are lighthearted and fun."
result = qa({"query": query})
print(result['result'])


Based on your request, I'd like to recommend the following three comedy anime that are lighthearted and fun:

1. **Kaminari Boy Pikkari★bee**: This anime is a comedy about a mischievous boy, with each episode containing two stories. Its adventurous and comedic tone makes it a great fit for those looking for a lighthearted watch.
2. **Pugyuru**: This anime is set in a world of absurdity, featuring a high school girl and her encounters with strange and fantastical characters, such as a snow fairy and a talking squid. Its comedic and fantastical elements make it a fun and entertaining watch.
3. **Shiawase no Katachi**: As a gag anime, **Shiawase no Katachi** is likely to deliver a lighthearted and humorous experience, with a focus on comedic storytelling and situations. Its simple yet amusing premise makes it a great choice for those seeking a fun and carefree anime.

All three of these anime seem to prioritize comedy and lightheartedness, making them suitable recommendations for your req

**Second Template**

In [ ]:
from langchain.prompts import PromptTemplate

template_prefix = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka. 
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime, 
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya. 
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'male')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: male
Question: {question}
Your response:


In [ ]:
PROMPT  =  PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Which anime would you suggest for someone new to anime?"
result = qa({"query": query})
print(result['result'])

For someone new to anime, I'd suggest "Clannad". It's a great introduction to the world of anime, with a mix of genres like comedy, drama, romance, and slice of life. The story is relatable, and the characters are well-developed and easy to follow. The anime also explores themes of friendship, love, and personal growth, which are common in many anime series. Additionally, "Clannad" is a well-known and highly-regarded anime, so it's a great representation of the medium as a whole. It's also a good starting point because it's not too complex or overwhelming, making it easy for a new viewer to get into.


In [ ]:
import gradio as gr
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate

# Load anime dataset
anime = pd.read_csv("/content/anime-filtered.csv")
anime = anime.dropna()
anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)

# Save updated dataset
anime[['combined_info']].to_csv('anime_updated.csv', index=False)
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()

# Text splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Create embeddings
embeddings = HuggingFaceEmbeddings()
docssearch = Chroma.from_documents(texts, embeddings)
retriever = docssearch.as_retriever()

# Define LLM
llm = ChatGroq(api_key="gsk_NHgqXXXx3sxGksbu5u3aWGdyb3FYXj8BKec5Sk29Jn2pkD83RAtb", model="allam-2-7b", temperature=0)

# Custom Prompt Template
template = """Anda adalah sistem pemberi rekomendasi film yang membantu pengguna menemukan anime yang sesuai dengan preferensi mereka. 
Gunakan potongan konteks berikut untuk menjawab pertanyaan di akhir. Untuk setiap pertanyaan, sarankan tiga anime, 
dengan deskripsi singkat tentang plot dan alasan mengapa pengguna mungkin menyukainya. 
Jika Anda tidak tahu jawabannya, katakan saja Anda tidak tahu, jangan mencoba mengarang jawaban.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa_chain({'query': query})
    return result['result']

# Gradio interface
theme = "Respair/Shiki@1.2.1"

iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,

)

if __name__ == "__main__":
    iface.launch()


ModuleNotFoundError: No module named 'gradio_doki'